In [3]:
import json
import os
from datetime import datetime

import weaviate
import pandas as pd
from snowflake.snowpark import Session

/opt/conda/lib/python3.11/site-packages/snowflake/snowpark/session.py:22: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/opt/conda/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('snowflake')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


testing...


In [3]:
class_obj_path = 'class_obj.json' # Specify the path to your class object JSON file
source_table = 'DESCRIBER_RESULTS'
class_name = 'Product' # Name of entity type in weaviate

In [5]:
# Make snowpark session
def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'],
    'port': os.environ['SNOWFLAKE_PORT'],
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'],
    'authenticator': 'oauth',
    'token': get_token(),
    'role': os.environ['SNOW_ROLE'],
    'warehouse': os.environ['SNOW_WAREHOUSE'],
    'database': os.environ["SNOW_DATABASE"],
    'schema': os.environ["SNOW_SCHEMA"]
}

session = Session.builder.configs(connection_params).create()

In [4]:
# Make connection to weaviate instance in other service
client = weaviate.Client(
   url = os.environ['WEAVIATE_URL'],  
 )

# Open and read the JSON file
with open(class_obj_path, 'r') as json_file:
    # Load the JSON data into a Python dictionary
    class_obj = json.load(json_file)

# Create schema in weaviate
client.schema.delete_all()
client.schema.create_class(class_obj)

cols = [i.get('name') for i in class_obj['properties']]
df = session.table(source_table).select(*cols).to_pandas()

def dataloader():
    """Iterator to load rows of pandas dataframe as dictionary"""
    import pandas as pd
    for _, row in df.iterrows():
        yield row.to_dict()

# Load data into weaviate in batches
print('Batched loading into weaviate started...')
now = datetime.now()
with client.batch as batch:
    batch.batch_size = 1000
    batch.dynamic = True
    for i, data_obj in enumerate(dataloader()):
        batch.add_data_object(
            data_obj,
            class_name,
        )
print('Weaviate loading completed.')
print(f'Execution time: {datetime.now() - now}.')